In [27]:
import numpy as np

def is_int(a):
	a = np.around(a,5)
	a = (a - np.floor(a))
	print(a)
	print(np.sum(a))
	if(np.sum(a)>0.000001):
		return False
	else:
		return True

def dual(A,B_i,solutions,xs):
	it = 0
	while True:
		it+=1
		print("Passo dual: ", it)
		print(A)
		# critério de parada
		if (A[:,len(A[0])-1] >= 0).all():
			if (A[0,:len(A[0])-2] >= 0).all():
				solutions.append(A)
				return
			elif (min(A[len(A)-1,:len(A[0])-2]) > -0.000001):
				return
			else:
				simplex_path(A,xs,B_i,"integer",solutions)
				return 
		# critério de infactibilidade 

		c_j = A[0][1:len(A[0])-2].copy()
		i = 0
		best = -1000000000
		best_i = 0
		while i < len(c_j):
			if A[len(A)-1,i+1] < 0:
				c_j[i] = c_j[i]/A[len(A)-1,i+1]
			if c_j[i] > best and c_j[i]<0:
				best = c_j[i]
				best_i = i+1
			i+=1
		A[len(A)-1,:] = A[len(A)-1,:]/A[len(A)-1,best_i]
		row_i = 0
		while(row_i < len(A)-1):
			A[row_i,:] = A[row_i,:] -A[row_i,best_i]*A[len(A)-1,:]
			row_i+=1
		B_i = []
		j = 1
		while j < len(A[0]):
			if abs(A[0][j])<= 0.0000001:
				B_i.append(j)
			j+=1

def new_branch(A,id_element,B_i,solutions,xs):
	i = np.where(A[1:,id_element]==1)[0][0] + 1
	new_restriction = np.floor(A[i,len(A[0])-1])
	A2 = np.zeros((len(A)+1,len(A[0])+1))

	A2[:len(A),:len(A[0])-1] = A[:,:len(A[0])-1]
	A2[:len(A),len(A2[0])-1] = A[:,len(A[0])-1]
	A2[len(A2)-1,id_element] = 1
	A2[len(A2)-1,len(A2[0])-2] = 1
	A2[len(A2)-1,len(A2[0])-1] = new_restriction

	A2[len(A2)-1,:] = A2[len(A2)-1,:] - A2[id_element,:] 

	B_i.append(len(A2[0])-2)

	dual(A2.copy(),B_i,solutions,xs)


	new_restriction+=1

	A2[len(A2)-1,:] = np.zeros(len(A2[0]))
	A2[len(A2)-1,id_element] = 1
	A2[len(A2)-1,len(A2[0])-2] = -1
	A2[len(A2)-1,len(A2[0])-1] = new_restriction
	A2[len(A2)-1,:] = A2[len(A2)-1,:] - A2[id_element,:] 
	A2[len(A2)-1,:] = -A2[len(A2)-1,:]

	dual(A2.copy(),B_i,solutions,xs)
	return 


def simplex_path(A,xs,B_i,sol,solutions):
	iteration = 0
	while True:
		iteration+=1
		ci = A[0] 
		print("Simplex Tableau: ", iteration)
		print(A)
		minn = np.min(ci[:len(ci)-1])
		if minn >= 0:
			if sol == "linear":
				return A
			if sol == "integer":
				cond = is_int(A[:,len(A[0])-1]) 
				print(cond)
				if cond and A[0,len(A[0])-1] != 0:
					return A
				else:
					for i in B_i:
						for j in xs:
							if(i==j):
								not_int = i
								break
					new_branch(A,not_int,B_i,solutions,xs)
					m = solutions[0]
					for i in solutions:
						if i[0,len(i[0])-1] >= m[0,len(i[0])-1]:
							m = i 
					return m
			else:
				print("solver não é válida")
				return 

		cond = (ci == minn)
		col = np.where(cond)[0][0]
		j = 0
		while col in B_i:
			col = list(range(1,len(ci)))[j]
			#print(index1)
			j+=1
		j = 1
		b_j = np.zeros(len(A))
		A_col = A[:,col] 
		#print(A_index1)
		# print("foi foi foi\n",index1)
		if (A_col <= 0).all():
			print("O problema é ilimitado")
			if solutions != []:
				m = solutions[0]
				for i in solutions:
					if i[0,len(i[0])-1] >= m[0,len(i[0])-1]:
						m = i 
					return m
			else:
				return 
		while j < len(A):
			if A[j,col] <= 0:
				b_j[j] = np.inf
			else:
				b_j[j] = A[j,len(ci)-1]/A[j,col]
			j+=1
		#print(b_j)
		row = np.min(b_j[1:])
		cond = (b_j == row)
		row = np.where(cond)[0][0]
		#print(index2)
		A[row] = A[row]/A[row,col]
		j = 0
		while j < len(A):
			if j == row or A[j,col]==0:
				j+=1
				continue
			A[j] = A[j] - A[j,col]*A[row]
			j+=1
		#print(A)
		B_i = []
		j = 1
		while j < len(A[0]):
			if abs(A[0][j])<= 0.0000001:
				B_i.append(j)
			j+=1

def tableau_form(A,b,c,B_i,t):
	xs = []
	for i in range(len(c)):
		if c[i] != 0:
			xs.append(i+1)
	a1 = np.zeros((len(A),1))
	B_i = list(map(lambda x: x+1, B_i))
	b = np.array([b]).T
	A = np.concatenate([a1, A, b], axis=1)
	c = np.append(c,0)
	if t == "max":
		A = np.concatenate([-np.array([np.hstack((-1,c))]),A],axis=0)
	else:
		A = np.concatenate([np.array([np.hstack((-1,c))]),A],axis=0)
	return A,xs,B_i


def tableau(A,b,c,B_i,t="max",sol="linear"):
	A,xs,B_i = tableau_form(A,b,c,B_i,t)
	solutions = []
	A = simplex_path(A,xs,B_i,sol,solutions)
	return A
	

c = np.array([3,2,0,0])

A = np.array([[3,1,1,0],[1,2,0,1]])

b = np.array([2,5])

A_bar = tableau(A,b,c,[2,3],sol="integer")

print("\nA matriz A final é\n",A_bar)


# c3 = np.array([1,5,0,0])

# A3 = np.array([[-4,3,1,0],[3,2,0,1]])

# b3 = np.array([6,18])

# print(tableau(A3,b3,c3,[2,3],sol="integer"))

Simplex Tableau:  1
[[ 1. -3. -2.  0.  0.  0.]
 [ 0.  3.  1.  1.  0.  2.]
 [ 0.  1.  2.  0.  1.  5.]]
Simplex Tableau:  2
[[ 1.          0.         -1.          1.          0.          2.        ]
 [ 0.          1.          0.33333333  0.33333333  0.          0.66666667]
 [ 0.          0.          1.66666667 -0.33333333  1.          4.33333333]]
Simplex Tableau:  3
[[ 1.  3.  0.  2.  0.  4.]
 [ 0.  3.  1.  1.  0.  2.]
 [ 0. -5.  0. -2.  1.  1.]]
[0. 0. 0.]
0.0
True

A matriz A final é
 (4.0, array([[ 1.,  3.,  0.,  2.,  0.,  4.],
       [ 0.,  3.,  1.,  1.,  0.,  2.],
       [ 0., -5.,  0., -2.,  1.,  1.]]))
